In [42]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk

In [39]:
csv_path = 'lg10_eval.csv'

df = pd.read_csv(csv_path, delimiter=',')

# drop the last column
df = df.iloc[:, :-1]

In [13]:
print(df.columns)
print(df.head())

Index(['FileName', 'EntityName', 'Layer', 'ID', 'Bulge', 'X', 'Y'], dtype='object')
             FileName    EntityName    Layer  ID Bulge       X       Y
0  Q881LG10SE1430.DWG  AcDbPolyline  TYPE_T1   1   nil -5.0676   83.35
1  Q881LG10SE1430.DWG  AcDbPolyline  TYPE_T1   1   nil -4.0651   83.35
2  Q881LG10SE1430.DWG  AcDbPolyline  TYPE_T1   1   nil -4.0651  114.85
3  Q881LG10SE1430.DWG  AcDbPolyline  TYPE_T1   1   nil -5.0676  114.85
4  Q881LG10SE1430.DWG  AcDbPolyline  TYPE_T1   1   nil -5.0676   83.35


In [14]:
def calculate_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [72]:
thickness_results = []
height_results = []
# Group the data by 'Layer' and 'ID' to find continuous walls
grouped_data = df.groupby(['Layer', 'ID'])

for (layer, wall_id), group in grouped_data:
    # Keep the points in the original order (no sorting)
    group_original = group.reset_index(drop=True)
    
    # Check if there are more than 4 points (indicating a duplicate)
    if len(group_original) > 4:
        # Remove the last point (assumed to be a duplicate of the first point)
        group_original = group_original.iloc[:-1]

    # Ensure we still have exactly 4 points to form the boundary of the wall
    if len(group_original) == 4:
        # Extract the first four points (X, Y)
        x_coords = group_original['X'].values
        y_coords = group_original['Y'].values
        
        # Calculate distances between consecutive points
        edge1 = calculate_distance(x_coords[0], y_coords[0], x_coords[1], y_coords[1])
        edge2 = calculate_distance(x_coords[1], y_coords[1], x_coords[2], y_coords[2])
        edge3 = calculate_distance(x_coords[2], y_coords[2], x_coords[3], y_coords[3])
        edge4 = calculate_distance(x_coords[3], y_coords[3], x_coords[0], y_coords[0])
        
        # Get the smallest edge as the wall thickness
        thickness = min(edge1, edge2, edge3, edge4)
        thickness = round(thickness, 3)
        
        # Get the largest edge as the wall height
        height = max(edge1, edge2, edge3, edge4)
        height = round(height, 3)

        print(f"Layer: {layer}, ID: {wall_id}, Thickness: {thickness}, Height: {height}")
        
        # Append the result for this wall
        thickness_results.append({'Layer': layer, 'ID': wall_id, 'Thickness': thickness})
        height_results.append({'Layer': layer, 'ID': wall_id, 'Height': height})

# Convert results to DataFrames
thickness_df = pd.DataFrame(thickness_results)
height_df = pd.DataFrame(height_results)

# Final thickness and height results
final_thickness_results = []
final_height_results = []

def handle_inconsistent_layers(layer, values, value_type):
    """Display a Tkinter window for the user to select the correct thickness or height"""
    root = tk.Tk()
    root.title(f"Select correct {value_type} - {layer}")

    label = tk.Label(root, text=f"Layer {layer} has inconsistent {value_type}: {values}")
    label.pack(pady=10)

    selected_value = tk.DoubleVar()

    for value in values:
        cleaned_value = float(str(value).replace('[','').replace(']',''))
        ttk.Radiobutton(root, text=str(cleaned_value), variable=selected_value, value=cleaned_value).pack(anchor='w')

    def confirm_selection():
        if value_type == 'thickness':
            final_thickness_results.append({'Layer': layer, 'Thickness': selected_value.get()})
        elif value_type == 'height':
            final_height_results.append({'Layer': layer, 'Height': selected_value.get()})
        root.destroy()

    confirm_button = ttk.Button(root, text="Confirm", command=confirm_selection)
    confirm_button.pack(pady=10)

    root.mainloop()

# Check thickness consistency
grouped_by_layer_thickness = thickness_df.groupby('Layer')
for layer, group in grouped_by_layer_thickness:
    unique_thicknesses = group['Thickness'].unique()
    if np.all(np.isclose(unique_thicknesses, unique_thicknesses[0])):
        final_thickness_results.append({'Layer': layer, 'Thickness': group['Thickness'].iloc[0]})
    else:
        handle_inconsistent_layers(layer, unique_thicknesses, 'thickness')

# Check height consistency
grouped_by_layer_height = height_df.groupby('Layer')
for layer, group in grouped_by_layer_height:
    unique_heights = group['Height'].unique()
    if np.all(np.isclose(unique_heights, unique_heights[0])):
        final_height_results.append({'Layer': layer, 'Height': group['Height'].iloc[0]})
    else:
        handle_inconsistent_layers(layer, unique_heights, 'height')

# Convert the final results into DataFrames for display
final_thickness_df = pd.DataFrame(final_thickness_results)
final_height_df = pd.DataFrame(final_height_results)

 # turn the dataframe into a dictionary
result_dir = {}
for index, row in final_thickness_df.iterrows():
    result_dir[row['Layer']] = {'Thickness': row['Thickness']}   
for index, row in final_height_df.iterrows():
    result_dir[row['Layer']]['Height'] = row['Height'] 

print(result_dir)

Layer: TYPE_T1, ID: 2, Thickness: 1.0, Height: 31.5
Layer: TYPE_T1, ID: 4, Thickness: 1.0, Height: 31.5
Layer: TYPE_T1A, ID: 2, Thickness: 1.0, Height: 31.5
Layer: TYPE_T2, ID: 1, Thickness: 0.8, Height: 18.5
{'TYPE_T1': {'Thickness': 1.0, 'Height': 31.5}, 'TYPE_T1A': {'Thickness': 1.0, 'Height': 31.5}, 'TYPE_T2': {'Thickness': 0.8, 'Height': 18.5}}


In [70]:
print(final_thickness_df)
print(final_height_df)

      Layer  Thickness
0   TYPE_T1        1.0
1  TYPE_T1A        1.0
2   TYPE_T2        0.8
      Layer  Height
0   TYPE_T1    31.5
1  TYPE_T1A    31.5
2   TYPE_T2    18.5


In [68]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk

# Simulated thickness_results DataFrame
thickness_results = [
    {'Layer': 'TYPE_E1', 'Thickness': 1.2},
    {'Layer': 'TYPE_E1', 'Thickness': 1.2},
    {'Layer': 'TYPE_E2', 'Thickness': 0.8},
    {'Layer': 'TYPE_E2', 'Thickness': 0.8002},
    {'Layer': 'TYPE_S2-2', 'Thickness': 1.3},
    {'Layer': 'TYPE_S2-2', 'Thickness': 1.1}
]

# Simulated height_results DataFrame
height_results = [
    {'Layer': 'TYPE_E1', 'Height': 2.5},
    {'Layer': 'TYPE_E1', 'Height': 2.5},
    {'Layer': 'TYPE_E2', 'Height': 2.0},
    {'Layer': 'TYPE_E2', 'Height': 2.0001},
    {'Layer': 'TYPE_S2-2', 'Height': 2.7},
    {'Layer': 'TYPE_S2-2', 'Height': 2.8}
]

# Convert results to DataFrames
thickness_df = pd.DataFrame(thickness_results)
height_df = pd.DataFrame(height_results)

# Final thickness and height results
final_thickness_results = []
final_height_results = []

def handle_inconsistent_layers(layer, values, value_type):
    """Display a Tkinter window for the user to select the correct thickness or height"""
    root = tk.Tk()
    root.title(f"Select correct {value_type} - {layer}")

    label = tk.Label(root, text=f"Layer {layer} has inconsistent {value_type}: {values}")
    label.pack(pady=10)

    selected_value = tk.DoubleVar()

    for value in values:
        cleaned_value = float(str(value).replace('[','').replace(']',''))
        ttk.Radiobutton(root, text=str(cleaned_value), variable=selected_value, value=cleaned_value).pack(anchor='w')

    def confirm_selection():
        if value_type == 'thickness':
            final_thickness_results.append({'Layer': layer, 'Thickness': selected_value.get()})
        elif value_type == 'height':
            final_height_results.append({'Layer': layer, 'Height': selected_value.get()})
        root.destroy()

    confirm_button = ttk.Button(root, text="Confirm", command=confirm_selection)
    confirm_button.pack(pady=10)

    root.mainloop()

# Check thickness consistency
grouped_by_layer_thickness = thickness_df.groupby('Layer')
for layer, group in grouped_by_layer_thickness:
    unique_thicknesses = group['Thickness'].unique()
    if np.all(np.isclose(unique_thicknesses, unique_thicknesses[0])):
        final_thickness_results.append({'Layer': layer, 'Thickness': group['Thickness'].iloc[0]})
    else:
        handle_inconsistent_layers(layer, unique_thicknesses, 'thickness')

# Check height consistency
grouped_by_layer_height = height_df.groupby('Layer')
for layer, group in grouped_by_layer_height:
    unique_heights = group['Height'].unique()
    if np.all(np.isclose(unique_heights, unique_heights[0])):
        final_height_results.append({'Layer': layer, 'Height': group['Height'].iloc[0]})
    else:
        handle_inconsistent_layers(layer, unique_heights, 'height')

# Convert the final results into DataFrames for display
final_thickness_df = pd.DataFrame(final_thickness_results)
final_height_df = pd.DataFrame(final_height_results)

print("Final Thickness Results:")
print(final_thickness_df)
print("\nFinal Height Results:")
print(final_height_df)

Final Thickness Results:
       Layer  Thickness
0    TYPE_E1        1.2
1    TYPE_E2        0.0
2  TYPE_S2-2        0.0

Final Height Results:
       Layer  Height
0    TYPE_E1     2.5
1    TYPE_E2     0.0
2  TYPE_S2-2     0.0
